In [ ]:
#Importing required libraries
import requests
import locale
import csv
import pandas as pd
import glob

In [ ]:
#Using tmdb Genre API, extracting list of movie genres available 
data_file = open('data_genre.csv', 'w')
csv_writer = csv.writer(data_file)
genre = 'https://api.themoviedb.org/3/genre/movie/list?api_key=308b04f2b1399d2700c3d6209623e241&language=en-US'
response = requests.get(genre)
response_json = response.json()
results = response_json['genres']
csv_writer.writerow(results[0].keys())
for result in results:
  print(result)
  csv_writer.writerow(result.values())
data_file.close()

In [ ]:
#for all 19 genres, 200 pages of results were extracted. With 20 results per page, a total 79,000 results were expected however each genre did not have complete 4,000 movies, so overall approximately 75,000 results were extract
genres = {'28','12','16','35','80','99','18','10751','14','36','27','10402','9648','10749','878','10770','53','10752','37'}
for genre in genres:
  data_file = open('data_file_genre_' + genre + '.csv', 'w')
  csv_writer = csv.writer(data_file)

  url1 = 'https://api.themoviedb.org/3/discover/movie?api_key=308b04f2b1399d2700c3d6209623e241&language=en&with_original_language=en&include_adult=true&with_genres=' + genre + '&page='

  for x in range(200):
    url = url1+str(x+1)
    response = requests.get(url)
    response_json = response.json()
    results = response_json['results']
    print(genre + ' ' + str(x))
    if(x==0):
      csv_writer.writerow(results[0].keys())
    for result in results:
      csv_writer.writerow(result.values())
  data_file.close()

In [ ]:
#Merge each genre file and de-dupe to create final Movie list
all_files = glob.glob('*.csv')
df_from_each_file = (pd.read_csv(f) for f in all_files)
concatenated_df   = pd.concat(df_from_each_file, ignore_index=True)
concatenated_df['id'].unique()
concatenated_df.drop_duplicates(subset=['id'],keep='first').to_csv('Movie_list.csv')